# Pandas Data Workflow: End-to-End Guide

This notebook demonstrates how to use core and advanced pandas functionality for a full analytics workflow: data ingestion, exploration, cleaning, transformation, feature engineering, and preparing design matrices suitable for scikit-learn model training.



## Workflow Roadmap

1. Environment setup and consistent plotting/theme configuration
2. Data ingestion from CSV (penguins dataset) and initial inspection
3. Core pandas operations: indexing, selection, filtering, sorting
4. Exploratory data analysis (EDA): descriptive statistics, grouping, aggregation, reshaping, visualization hooks
5. Data quality: missing values, outliers, duplicates, type casting
6. Feature engineering: dates, categorical encoding, numerical transformations
7. Preparing train-ready feature (`X`) and target (`y`) matrices, including scikit-learn compatible pipelines
8. Lightweight model training example to validate the dataset hand-off to scikit-learn
9. Saving artifacts and documenting reusable utilities



In [ ]:
import sys
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 3)
sns.set_theme(style="whitegrid", context="talk")

REPO_ROOT = pathlib.Path("../../..").resolve()
RAW_DATA_URL = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"



In [ ]:
import importlib
import subprocess

def ensure(package: str):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ("pyarrow",):
    ensure(pkg)



## 1. Data Ingestion & Initial Inspection

We use the penguins dataset, emphasizing how pandas reads remote CSV data, persists raw copies locally when needed, and inspects dtypes/shape/metadata.



In [ ]:
raw_path = REPO_ROOT / "data" / "penguins_raw.csv"
raw_path.parent.mkdir(parents=True, exist_ok=True)

df_raw = pd.read_csv(RAW_DATA_URL)
df_raw.to_csv(raw_path, index=False)

print(f"Rows: {df_raw.shape[0]}, Columns: {df_raw.shape[1]}")
df_raw.head()



In [ ]:
df_raw.info()



## 2. Core Pandas Operations

Below we demonstrate slicing, boolean masks, sorting, selecting columns, and leveraging `assign`, `pipe`, and `query` for expressive transformations.



In [ ]:
selected_cols = ["species", "island", "bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g", "sex"]
df = df_raw[selected_cols].copy()

adelie_mask = df["species"].eq("Adelie")
adelie_sorted = df.loc[adelie_mask].sort_values(by="bill_length_mm", ascending=False)
adelie_sorted.head(3)



In [ ]:
def calc_bill_ratio(frame: pd.DataFrame) -> pd.DataFrame:
    return frame.assign(bill_ratio=frame["bill_length_mm"] / frame["bill_depth_mm"])

(df
 .dropna(subset=["bill_length_mm", "bill_depth_mm"])
 .pipe(calc_bill_ratio)
 .query("bill_ratio > 2.5")
 .head()
)



## 3. Exploratory Data Analysis (EDA)

Use `describe`, `value_counts`, `groupby`, and visualization-ready summaries for deeper understanding.



In [ ]:
df.describe(include="all").T



In [ ]:
grouped = (df
           .dropna(subset=["body_mass_g"])
           .groupby(["species", "sex"], observed=True)
           .agg(
               mean_mass=("body_mass_g", "mean"),
               sd_mass=("body_mass_g", "std"),
               n=("body_mass_g", "size")
           )
           .reset_index()
          )
grouped



In [ ]:
pivot = pd.pivot_table(
    grouped,
    values="mean_mass",
    index="species",
    columns="sex",
    aggfunc="mean"
)
pivot



In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(data=df, x="body_mass_g", hue="species", kde=True)
plt.title("Body Mass Distribution by Species")
plt.tight_layout()



## 4. Data Quality & Cleaning

Identify missingness, duplicate handling, outlier rules, and dtype normalization.



In [ ]:
missing_profile = df.isna().mean().mul(100).round(2)
missing_profile



In [ ]:
duplicates = df.duplicated(subset=["species", "island", "bill_length_mm", "bill_depth_mm", "body_mass_g"])
duplicates.sum()



In [ ]:
clean_df = df.drop_duplicates().copy()

numeric_cols = clean_df.select_dtypes(include="number").columns
clean_df[numeric_cols] = clean_df[numeric_cols].apply(lambda col: col.fillna(col.median()))

clean_df["sex"] = clean_df["sex"].fillna("Unknown").astype("category")
clean_df["species"] = clean_df["species"].astype("category")
clean_df.dtypes



## 5. Feature Engineering

Demonstrate numerical scaling, binning, categorical encoding, interaction features, and tidy reshaping patterns.


  


In [ ]:
feature_df = (clean_df
              .assign(
                  body_mass_kg=lambda d: d["body_mass_g"] / 1000,
                  flipper_to_bill=lambda d: d["flipper_length_mm"] / d["bill_length_mm"],
                  mass_bin=lambda d: pd.cut(d["body_mass_g"], bins=[2500, 4000, 5000, 6500], labels=["small", "medium", "large"])
              )
             )
feature_df.head()



In [ ]:
long_df = feature_df.melt(
    id_vars=["species", "island", "sex"],
    value_vars=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"],
    var_name="measurement",
    value_name="value"
)
long_df.head()



## 6. Train/Test Split & Modeling Prep

Define `X` (features) and `y` (labels), encode categoricals, scale numerics, and build a scikit-learn pipeline.



In [ ]:
TARGET = "species"
FEATURES = [col for col in feature_df.columns if col not in {TARGET}]

X = feature_df[FEATURES].copy()
y = feature_df[TARGET].copy()

numeric_features = X.select_dtypes(include=["number"]).columns.tolist()
categorical_features = X.select_dtypes(exclude=["number"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=200))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))



In [ ]:
ohe_feature_names = clf.named_steps["preprocess"].named_transformers_["cat"].named_steps["onehot"].get_feature_names_out(categorical_features)
print("Numeric features:", numeric_features)
print("Encoded categorical features:", list(ohe_feature_names)[:5], "...")



## 7. Persisting Processed Data

Save cleaned data and modeling splits for reproducibility.



In [ ]:
processed_dir = REPO_ROOT / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

feature_df.to_parquet(processed_dir / "penguins_features.parquet", index=False)
X_train.to_parquet(processed_dir / "penguins_X_train.parquet", index=False)
X_test.to_parquet(processed_dir / "penguins_X_test.parquet", index=False)
y_train.to_frame(name=TARGET).to_csv(processed_dir / "penguins_y_train.csv", index=False)
y_test.to_frame(name=TARGET).to_csv(processed_dir / "penguins_y_test.csv", index=False)

processed_dir



## Key Takeaways

- Pandas simplifies the entire analytics lifecycle: ingest, explore, clean, engineer, and persist datasets.
- Consistent schemas and dtype management ensure painless hand-offs to scikit-learn pipelines.
- Saving processed data and reusing transformation utilities preserves reproducibility and accelerates iteration.

